In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA
%matplotlib inline
import random
from sklearn import preprocessing
import scipy.sparse as sparse
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import graphlab as gl

## Overview

This Notebook allows for easy use of Turi's impressive GraphLab Library.  This notebook will create a Collaborative Filtering Recommendation System that recommends books with an elegant user interface that includes the product images.

In [2]:
filename = "datasets/reviews.csv"
df = pd.read_csv(filename)

In [3]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [4]:
data = gl.SFrame.read_csv(filename)

This non-commercial license of GraphLab Create for academic use is assigned to atsoroka@g.harvard.edu and will expire on December 03, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1481129403.log


Unable to parse line "2827,B001NIZLZM,"[21, 23]",1,"This is analogous to those straight-to-video movies that end up in the $3 bin at Big Lots. Somehow, they are produced, written and made; but in so many cases, you end up wondering...what were they thinking?In this case, the au..."

Unable to parse line "28648,B004443YVK,"[1, 1]",5,Unbelievable how depraved the human spirit can become and yet such goodness and compassion prevails at the same time.  Thank God he can overcome our failures.,"04 26, 2013",A1KNXLIXUTUY0L,"Carol  berry"

Unable to parse line "Carol berry",The terror continues by so does the good.,1366934400"

...

Read 463856 lines. Lines per second: 62583.4

Read 820541 lines. Lines per second: 62474.7

411 lines failed to parse correctly

Finished parsing file /Users/anthonysoroka/Documents/Academic/HarvardCSE/AC209/Project/Milestone3/datasets/reviews.csv

Parsing completed. Parsed 982409 lines in 15.5109 secs.

In [5]:
data

X1,asin,helpful,overall,reviewText,reviewTime,reviewerID
0,B000F83SZQ,"[0.0, 0.0]",5,I enjoy vintage books andmovies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J
1,B000F83SZQ,"[2.0, 2.0]",4,This book is a reissue ofan old one; the author ...,"01 6, 2014",AN0N05A9LIJEQ
2,B000F83SZQ,"[2.0, 2.0]",4,This was a fairlyinteresting read. It ...,"04 4, 2014",A795DMNCJILA6
3,B000F83SZQ,"[1.0, 1.0]",5,I'd never read any of theAmy Brewster mysteries ...,"02 19, 2014",A1FV0SX13TWVXQ
4,B000F83SZQ,"[0.0, 1.0]",4,"If you like period pieces- clothing, lingo, you ...","03 19, 2014",A3SPTOKDG7WBLN
5,B000F83SZQ,"[0.0, 0.0]",4,A beautiful in-depthcharacter description ...,"05 26, 2014",A1RK2OCZDSGC6R
6,B000F83SZQ,"[0.0, 0.0]",4,I enjoyed this one thoI'm not sure why it's ...,"06 10, 2014",A2HSAKHC3IBRE6
7,B000F83SZQ,"[1.0, 1.0]",4,Never heard of AmyBrewster. But I don't ...,"03 22, 2014",A3DE6XGZ2EPADS
8,B000FA64PA,"[0.0, 0.0]",5,Darth Maul working undercloak of darkness ...,"10 11, 2013",A1UG4Q4D3OAH3A
9,B000FA64PA,"[0.0, 0.0]",4,This is a short storyfocused on Darth Maul's ...,"02 13, 2011",AQZH7YTWQPOBE


In [6]:
items = data.groupby('asin', gl.aggregate.MEAN('overall'))

In [7]:
items

asin,Avg of overall
B00BRLB1LS,4.83333333333
B00DSSEHW8,4.89473684211
B00HFHEC9Q,4.83333333333
B00HFIZ3E8,4.0
B00F1Y9M04,4.25
B001RV3OLS,4.6
B00A3YDWW6,4.625
B00ISBMQ5A,4.83333333333
B00C2FCC0M,4.22222222222
B00GH9OOQE,3.83333333333


In [8]:
items = items.add_row_number('item_id')

In [9]:
items

item_id,asin,Avg of overall
0,B00BRLB1LS,4.83333333333
1,B00DSSEHW8,4.89473684211
2,B00HFHEC9Q,4.83333333333
3,B00HFIZ3E8,4.0
4,B00F1Y9M04,4.25
5,B001RV3OLS,4.6
6,B00A3YDWW6,4.625
7,B00ISBMQ5A,4.83333333333
8,B00C2FCC0M,4.22222222222
9,B00GH9OOQE,3.83333333333


In [10]:
urls = gl.SFrame.read_csv('datasets/meta_Kindle_Store.csv')

Unable to parse line "B003QP4GPQ,"Alan Simon is the author of several historical novels set in the mid-20th century: TheFirst Christmas of the War,Thanksgiving, 1942, andUnfinished Business. He is also the author of the serialized novelGettysburg, 1913: A Novel of the Great Reu..."

Unable to parse line "Battle of Gettysburg and the 100th anniversary of the 1913 &#x22;Great Reunion&#x22; that was held 50 years"

Unable to parse line "later and was attended by more than 50,000 aging Civil War veterans.Alan is also the author of the memoirClemente: Memories of a Once-Young Fan - Four Birthdays, Three World Series, Two Holiday Steelers Games, and One Bar Mitzvah,"

Unable to parse line " published in honor and memory of the 40th anniversary of baseball"

Unable to parse line "B002NGO5O6,"Applause for David Alexander's novel Brooklynese&#x22;From the"

...

Read 281980 lines. Lines per second: 18924.2

Read 396385 lines. Lines per second: 19383.5

90218 lines failed to parse correctly

Finished parsing file /Users/anthonysoroka/Documents/Academic/HarvardCSE/AC209/Project/Milestone3/datasets/meta_Kindle_Store.csv

Parsing completed. Parsed 427421 lines in 21.2491 secs.

In [11]:
urls

asin,description,price,imUrl,related
1603420304,In less time and for lessmoney than it takes to ...,7.69,http://ecx.images-amazon.com/images/I/51IEqPrF ...,"{'also_viewed':['B001OLRKLQ', ..."
B0002IQ15S,This universal DC adapterpowers/charges portable ...,19.99,http://ecx.images-amazon.com/images/I/21QFJM28 ...,"{'also_viewed':['B00511PS3C', ..."
B000F83SZQ,,0.0,http://ecx.images-amazon.com/images/I/51yLqHe% ...,"{'also_bought':['B0080H1C0W', ..."
B000F83TEQ,,None,http://ecx.images-amazon.com/images/I/2136NBNV ...,"{'also_bought':['B00IS81LFO', ..."
B000F83STC,,None,http://g-ecx.images-amazon.com/images/G/01/x-s ...,None
B000FA5RE4,Updated 2003 version withcorrections ...,1.99,http://ecx.images-amazon.com/images/I/21XTM6V0 ...,"{'also_viewed':['B001GX3ZWA', ..."
B000FA5NSO,,None,http://ecx.images-amazon.com/images/I/51N45B6j ...,"{'also_viewed':['B000FBF81K', ..."
B000FA5UXC,Madoc Tamlin is a manwith an unusual problem. ...,None,http://ecx.images-amazon.com/images/I/51q4iur5 ...,"{'also_bought':['B004SPL1I8', ..."
B000FA5T4W,,12.46,http://ecx.images-amazon.com/images/I/51eWyBr8 ...,"{'also_bought':['B004RKXHGS', ..."
B000FA5SHK,,None,http://ecx.images-amazon.com/images/I/51c7mqOR ...,None


In [14]:
items2 = items.join(urls, on='asin')

In [15]:
items2

item_id,asin,Avg of overall,description,price,imUrl
28192,B000F83SZQ,4.25,,0.0,http://ecx.images-amazon.com/images/I/51yLqHe% ...
17688,B000FA64QO,3.8,,5.99,http://ecx.images-amazon.com/images/I/51h5NLeM ...
31608,B000FA64PA,4.2,,4.58,http://ecx.images-amazon.com/images/I/4138cT%2 ...
3563,B000FA64PK,4.375,,5.99,http://ecx.images-amazon.com/images/I/31Ov6Go% ...
23243,B000FBFMVG,4.33333333333,,1.99,http://ecx.images-amazon.com/images/I/31xnW4Cw ...
1044,B000FC1BN8,3.44444444444,,4.59,http://ecx.images-amazon.com/images/I/51WMe3%2 ...
13404,B000FC1TG2,5.0,"Tom Shanaghy was iron-tough, but he was no ...",5.12,http://ecx.images-amazon.com/images/I/41uIBiFz ...
39289,B000FC26RI,4.16666666667,The third installment inBell Tower's attracti ...,9.99,http://ecx.images-amazon.com/images/I/31zMSp69 ...
42578,B000FC2MB8,4.625,Major James Brionnebrought Dave Allard to ...,5.12,http://ecx.images-amazon.com/images/I/51JX4ARE ...
24389,B000FDJ0FS,2.46666666667,Few newspapers enjoy theprestige and authorit ...,None,


In [16]:
data2 = data.join(items2, on='asin')

In [17]:
data2

X1,asin,helpful,overall,reviewText,reviewTime,reviewerID
0,B000F83SZQ,"[0.0, 0.0]",5,I enjoy vintage books andmovies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J
1,B000F83SZQ,"[2.0, 2.0]",4,This book is a reissue ofan old one; the author ...,"01 6, 2014",AN0N05A9LIJEQ
2,B000F83SZQ,"[2.0, 2.0]",4,This was a fairlyinteresting read. It ...,"04 4, 2014",A795DMNCJILA6
3,B000F83SZQ,"[1.0, 1.0]",5,I'd never read any of theAmy Brewster mysteries ...,"02 19, 2014",A1FV0SX13TWVXQ
4,B000F83SZQ,"[0.0, 1.0]",4,"If you like period pieces- clothing, lingo, you ...","03 19, 2014",A3SPTOKDG7WBLN
5,B000F83SZQ,"[0.0, 0.0]",4,A beautiful in-depthcharacter description ...,"05 26, 2014",A1RK2OCZDSGC6R
6,B000F83SZQ,"[0.0, 0.0]",4,I enjoyed this one thoI'm not sure why it's ...,"06 10, 2014",A2HSAKHC3IBRE6
7,B000F83SZQ,"[1.0, 1.0]",4,Never heard of AmyBrewster. But I don't ...,"03 22, 2014",A3DE6XGZ2EPADS
8,B000FA64PA,"[0.0, 0.0]",5,Darth Maul working undercloak of darkness ...,"10 11, 2013",A1UG4Q4D3OAH3A
9,B000FA64PA,"[0.0, 0.0]",4,This is a short storyfocused on Darth Maul's ...,"02 13, 2011",AQZH7YTWQPOBE


In [18]:
users = data2.groupby(['reviewerName','reviewerID'], gl.aggregate.MEAN('overall'))

In [19]:
users

reviewerID,reviewerName,Avg of overall
ANGMPTBPTVTU1,Amazon Customer,4.0
A3LNJX5X6ZKYNW,Amazon Customer,4.875
A2QMIRJ8E6L4Y9,Bonnie B Moore,4.66666666667
A2MVGSLY3G7UFQ,Christine,5.0
ALC2AF0EUUEH2,Amazon Customer,5.0
A1TD0NXK1D5SO8,"Teresa Wilcox""Seasiren770"" ...",3.4375
A1Q384IOUQ95G2,Brandy,5.0
A1BF0QGC2YXG1P,mindy,4.4
A1HJ5FE8BJYRBQ,P.Simmons,3.88888888889
A2PL2EYSVSQAEO,fiction addict,4.42857142857


In [20]:
train, valid = gl.recommender.util.random_split_by_user(data2, 'reviewerID', 'item_id', max_num_users=None, item_test_proportion=0.4)

In [56]:
train

X1,asin,helpful,overall,reviewText,reviewTime,reviewerID
2,B000F83SZQ,"[2.0, 2.0]",4,This was a fairlyinteresting read. It ...,"04 4, 2014",A795DMNCJILA6
4,B000F83SZQ,"[0.0, 1.0]",4,"If you like period pieces- clothing, lingo, you ...","03 19, 2014",A3SPTOKDG7WBLN
6,B000F83SZQ,"[0.0, 0.0]",4,I enjoyed this one thoI'm not sure why it's ...,"06 10, 2014",A2HSAKHC3IBRE6
9,B000FA64PA,"[0.0, 0.0]",4,This is a short storyfocused on Darth Maul's ...,"02 13, 2011",AQZH7YTWQPOBE
11,B000FA64PA,"[0.0, 0.0]",4,Title has nothing to dowith the story. I did ...,"09 17, 2011",A2ZFR72PT054YS
12,B000FA64PA,"[0.0, 0.0]",3,Well written. Interestingto see Sideous (through ...,"12 31, 2013",A2QK1U70OJ74P
13,B000FA64PK,"[0.0, 0.0]",3,Troy Denning's novellaRecovery was originally ...,"03 15, 2012",A3SZMGJMV0G16C
15,B000FA64PK,"[0.0, 0.0]",5,I really enjoyed thebook. Had the normal ...,"01 2, 2014",A2EN84QHDRZLP2
18,B000FA64PK,"[0.0, 1.0]",5,This one promises to beanother good book. I ...,"01 27, 2014",A1ZT7WV0ZUA0OJ
20,B000FA64PK,"[0.0, 0.0]",5,"Excellent! Very wellwritten story, very ...","12 31, 2013",A2QK1U70OJ74P


In [57]:
valid

X1,asin,helpful,overall,reviewText,reviewTime,reviewerID
0,B000F83SZQ,"[0.0, 0.0]",5,I enjoy vintage books andmovies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J
1,B000F83SZQ,"[2.0, 2.0]",4,This book is a reissue ofan old one; the author ...,"01 6, 2014",AN0N05A9LIJEQ
3,B000F83SZQ,"[1.0, 1.0]",5,I'd never read any of theAmy Brewster mysteries ...,"02 19, 2014",A1FV0SX13TWVXQ
5,B000F83SZQ,"[0.0, 0.0]",4,A beautiful in-depthcharacter description ...,"05 26, 2014",A1RK2OCZDSGC6R
7,B000F83SZQ,"[1.0, 1.0]",4,Never heard of AmyBrewster. But I don't ...,"03 22, 2014",A3DE6XGZ2EPADS
8,B000FA64PA,"[0.0, 0.0]",5,Darth Maul working undercloak of darkness ...,"10 11, 2013",A1UG4Q4D3OAH3A
10,B000FA64PA,"[0.0, 0.0]",5,I think I have this onein both book and audio. ...,"01 27, 2014",A1ZT7WV0ZUA0OJ
14,B000FA64PK,"[0.0, 0.0]",5,I am not for sure on howmuch of a difference the ...,"05 12, 2013",A3H8PE1UFK04JZ
16,B000FA64PK,"[0.0, 0.0]",5,Great read enjoyed everyminute of it . I think ...,"10 29, 2013",A1UG4Q4D3OAH3A
17,B000FA64PK,"[4.0, 4.0]",3,"Another well writteneBook by Troy Denning, ...","04 16, 2009",A38Z3Q6DTDIH9J


In [58]:
model = gl.recommender.create(train, 'reviewerID', 'item_id')

Recsys training: model = item_similarity

Warning: Ignoring columns X1, asin, helpful, overall, reviewText, reviewTime, reviewerName, summary, unixReviewTime, Avg of overall, description, price, imUrl, related, categories, title, salesRank, brand;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 1m 0s                               | 100              | 58655           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 60.5732s

In [59]:
# Save the model for later use
model.save("my_model")

In [12]:
model = gl.load_model("my_model")

In [21]:
view = model.views.overview(
        validation_set=valid,
        user_data=users,
        user_name_column='reviewerName',
        item_data=items2,
        item_name_column='asin',
        item_url_column='imUrl')

recommendations finished on 1000/65599 queries. users per second: 2262.81

recommendations finished on 2000/65599 queries. users per second: 2387.81

recommendations finished on 3000/65599 queries. users per second: 2462.7

recommendations finished on 4000/65599 queries. users per second: 2499.16

recommendations finished on 5000/65599 queries. users per second: 2507.49

...

recommendations finished on 61000/65599 queries. users per second: 1731.11

recommendations finished on 62000/65599 queries. users per second: 1736.57

recommendations finished on 63000/65599 queries. users per second: 1739.57

recommendations finished on 64000/65599 queries. users per second: 1736.19

recommendations finished on 65000/65599 queries. users per second: 1733.92

In [22]:
### To view the interface

view.show()